# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import pydot as pydot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.express as px
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,6)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import initializers
from tensorflow import keras
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn import over_sampling
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import precision_recall_curve
import xgboost as xgb
from sklearn.model_selection import cross_validate, learning_curve, cross_val_predict
import keras_tuner as kt


# Define functions

In [ ]:
def hypertune_threshold(model, start, stop, X_test, y_test, pos, neg):
    threshold = np.linspace(start = start, stop = stop, num = 100)
    f1 = np.zeros(threshold.size)
    predictions = model.predict_proba(X_test)
    for i in np.arange(threshold.size):
        probs = np.where(predictions[:,pos]>=threshold[i], pos, neg)
        f1[i] = f1_score(np.ravel(y_test), probs, pos_label=pos)
    final_probs = np.where(predictions[:,pos]>=threshold[np.argmax(f1)], pos, neg)
    return print(threshold[np.argmax(f1)], f1_score(np.ravel(y_test), final_probs, pos_label=pos)), final_probs

def idx_cm(y_pred, y_test, pos, neg):
    fp_rows = []
    fn_rows = []
    tp_rows = []
    tn_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == pos and y_test[i] == neg:
            fp_rows.append(i)
        elif y_pred[i] == pos and y_test[i] == pos:
            tp_rows.append(i)
        elif y_pred[i] == neg and y_test[i] == pos:
            fn_rows.append(i)
        elif y_pred[i] == neg and y_test[i] == neg:
            tn_rows.append(i)
    fp_rows = pd.DataFrame(fp_rows)
    tp_rows = pd.DataFrame(tp_rows)
    fn_rows = pd.DataFrame(fn_rows)
    tn_rows = pd.DataFrame(tn_rows)
    cm = [fp_rows, tp_rows, fn_rows, tn_rows]
    return cm


# Read in data

In [ ]:
X_train = pd.read_pickle('x_train_lvl1.pkl')
X_test = pd.read_pickle('x_test_lvl1.pkl')
y_train = np.ravel(pd.read_pickle('y_train_lvl1.pkl'))
y_test = np.ravel(pd.read_pickle('y_test_lvl1.pkl'))
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train.info())

# Prepare data


In [ ]:
y_train = ~y_train+2
y_test = ~y_test+2

X_train = np.array(X_train)
X_test = np.array(X_test)

scaler = preprocessing.MinMaxScaler((0, 1)) # normalize data for neural network and logistic regression
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_scaled, y_train)

print('Training labels shape:', y_resampled.shape)

print('Training features shape:', X_resampled.shape)


# Define models

In [ ]:
LR = LogisticRegression(solver='newton-cholesky')
RF = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=5, n_estimators=1250,random_state=1)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_auc',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


METRICS = [
      keras.metrics.AUC(name='auc')
]


NN = Sequential()
NN.add(Dense(256, input_shape=(X_train.shape[-1],), activation='relu'))
NN.add(Dropout(0.2))
NN.add(Dense(112, input_shape=(X_train.shape[-1],), activation='relu'))
NN.add(Dropout(0.2))
NN.add(Dense(1, activation='sigmoid'))
# Compile model
NN.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics="keras.metrics.auc")

# Train and evaluate models using cv
The following code represents the neural network. For logistic regression and LR the model is changed.

In [ ]:
# Define the maximum train set size and the number of folds
max_train_size = len(X_train)
n_folds = 5

# Initialize lists to store train set sizes and evaluation scores for each fold
train_sizes = []
auc_roc_scores_test = []
auc_roc_scores_train = []
f1_scores_test = []
f1_scores_train = []
brier_scores_test = []
brier_scores_train = []
loss_scores_test = []
loss_scores_train = []

# Perform cross-validation on different train set sizes
for train_size in range(500, max_train_size+1, 1000):
    # Initialize the cross-validation object
    skf = StratifiedKFold(n_splits=n_folds)

    fold_auc_roc_scores_test = []
    fold_f1_scores_test = []
    fold_brier_scores_test = []
    fold_loss_scores_test = []
    fold_auc_roc_scores_train = []
    fold_f1_scores_train = []
    fold_brier_scores_train = []
    fold_loss_scores_train = []

    # Split the data into train and test sets using stratified cross-validation
    for train_index, test_index in skf.split(X_train[:train_size], y_train[:train_size]):
        X_train_fold, X_test_fold = X_train[:train_size][train_index], X_train[:train_size][test_index]
        y_train_fold, y_test_fold = y_train[:train_size][train_index], y_train[:train_size][test_index]

        # Normalize the data within the fold
        scaler = preprocessing.MinMaxScaler((0,1))
        X_train_fold = scaler.fit_transform(X_train_fold)
        X_test_fold = scaler.transform(X_test_fold)

        # Resample
        ros = RandomUnderSampler(random_state=42)
        train_resampled, y_resampled = ros.fit_resample(X_train_fold, y_train_fold)
        # Define and compile the neural network model
        model = Sequential()
        model.add(Dense(256, kernel_initializer = initializers.RandomNormal(stddev=0.01), input_shape=(X_train.shape[-1],), activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(112, input_shape=(X_train.shape[-1],), activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=METRICS)

        # Train the model
        model.fit(train_resampled, y_resampled, epochs = 100, batch_size=2048, validation_data=(X_test_fold, y_test_fold), callbacks=[early_stopping])

        # Make predictions on the test set
        y_pred_test = model.predict(X_test_fold)
        y_pred_train = model.predict(X_train_fold)

        # Calculate and store the AUC-ROC score
        auc_roc_test = roc_auc_score(y_test_fold, y_pred_test)
        auc_roc_train = roc_auc_score(y_train_fold, y_pred_train)
        fold_auc_roc_scores_test.append(auc_roc_test)
        fold_auc_roc_scores_train.append(auc_roc_train)

        # Calculate and store the F1 score
        y_pred_binary_test = np.round(y_pred_test)
        y_pred_binary_train = np.round(y_pred_train)
        f1_test = f1_score(y_test_fold, y_pred_binary_test)
        f1_train = f1_score(y_train_fold, y_pred_binary_train)
        fold_f1_scores_test.append(f1_test)
        fold_f1_scores_train.append(f1_train)

        # Calculate and store the Brier loss
        brier_loss_test = brier_score_loss(y_test_fold, y_pred_test)
        fold_brier_scores_test.append(brier_loss_test)
        brier_loss_train = brier_score_loss(y_train_fold, y_pred_train)
        fold_brier_scores_train.append(brier_loss_train)

        # Calculate and store the loss
        loss_test = log_loss(y_test_fold, y_pred_test)
        fold_loss_scores_test.append(loss_test)
        loss_train = log_loss(y_train_fold, y_pred_train)
        fold_loss_scores_train.append(loss_train)

    # Calculate the average scores across all folds for the current train set size
    average_auc_roc_test = np.mean(fold_auc_roc_scores_test)
    average_f1_test = np.mean(fold_f1_scores_test)
    average_brier_test = np.mean(fold_brier_scores_test)
    average_loss_test = np.mean(fold_loss_scores_test)
    average_auc_roc_train = np.mean(fold_auc_roc_scores_train)
    average_f1_train = np.mean(fold_f1_scores_train)
    average_brier_train = np.mean(fold_brier_scores_train)
    average_loss_train = np.mean(fold_loss_scores_train)

    # Append the train set size and average scores to the respective lists
    train_sizes.append(train_size)
    auc_roc_scores_test.append(average_auc_roc_test)
    f1_scores_test.append(average_f1_test)
    brier_scores_test.append(average_brier_test)
    loss_scores_test.append(average_loss_test)
    auc_roc_scores_train.append(average_auc_roc_train)
    f1_scores_train.append(average_f1_train)
    brier_scores_train.append(average_brier_train)
    loss_scores_train.append(average_loss_train)


# Plot training curves - training size vs loss & training size vs f1 & training size vs roc_auc & training size vs brier

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=auc_roc_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=auc_roc_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='ROC_AUC')

In [ ]:
fig, ax = plt.subplots()
ax = plt.semilogy(train_sizes, auc_roc_scores_test, label="test")
ax1 = plt.semilogy(train_sizes, auc_roc_scores_train, label="train")

plt.xlabel('Number of observations')
plt.legend()
# Give y axis label for the semilogy plot
plt.ylabel('ROC_AUC')

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=f1_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=f1_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='F1')

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=loss_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=loss_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='Logloss')

In [ ]:
fig, ax = plt.subplots()
ax= sns.lineplot(x=train_sizes, y=brier_scores_test, label = "test")
ax1 = sns.lineplot(x=train_sizes, y=brier_scores_train, label = "train")
ax.set(xlabel='Number of observations', ylabel='Brier score')

# Identify false negatives and false positives and save indices for comparison between models

In [ ]:
NN.fit(X_resampled, y_resampled, epochs=1000, batch_size=2048, verbose=2, validation_data = (X_test_scaled, y_test), callbacks = [early_stopping])
LR_fitted = LR.fit(X_resampled, y_resampled)
RF_fitted = RF.fit(X_resampled, y_resampled)
test_predictions_baseline = NN.predict(X_test_scaled, batch_size=2048)
yhat = np.where(test_predictions_baseline > 0.5, 1, 0)  #predictions for greatest class
cm_LR = idx_cm(LR_fitted.predict(X_test_scaled), y_test, 1, 0)
cm_RF = idx_cm(RF_fitted.predict(X_test_scaled), y_test, 1, 0)
cm_NN = idx_cm(yhat, y_test, 1, 0)

In [ ]:
print(cm_RF[0].shape, cm_RF[1].shape, cm_RF[2].shape, cm_RF[3].shape)

In [ ]:
print(cm_NN[0].shape, cm_NN[1].shape, cm_NN[2].shape, cm_NN[3].shape)

In [ ]:
print(cm_LR[0].shape, cm_LR[1].shape, cm_LR[2].shape, cm_LR[3].shape)

In [ ]:
import itertools

cm_list = [cm_LR, cm_RF, cm_NN]

def inner_join_cm(cm_list): #cm_model needs to be a list with
    inner_all = []
    for model1, model2 in itertools.combinations(cm_list, 2):
        inner = np.zeros((len(cm_list[0])))
        for i in range(len(cm_list[0])):
            inner[i] = pd.merge(model1[i],model2[i], how="inner", indicator="True").shape[0]
        inner_all.append(inner)
    return inner_all

def left_join_cm(cm_list):
    left_all = []
    for model1, model2 in itertools.combinations(cm_list, 2):
        left = []
        for i in range(len(cm_list[0])):
            left = pd.merge(model1[i],model2[i], how="left", indicator="True")
            idx = left[left["True"]=="left_only"]
            left_all.append(idx)
    return left_all

In [ ]:
inner_join_cm(cm_list)

In [ ]:
print(X_train.shape, X_test.shape) # 162.757 --> only about 10.000 observations that the models disagree on

# Plot ROC curves

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test, model.predict_proba(X_test_scaled)[:,1])

#create ROC curve
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Plot Precision recall curves

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, model.predict_proba(X_test_scaled)[:, 1])
fig, ax = plt.subplots()
ax.plot(recall, precision, color='blue')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()